# generates a dictonary from the xml for parsing a struct
https://docs.python.org/3.7/library/struct.html

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
#parse the xml
tree = ET.parse(r'../../../../description/aadc.description')
root = tree.getroot()

In [2]:
for child in root:
    print(child.tag, child.attrib, end='\n')
    for ch in child:
        print('\t'+ch.tag, ch.attrib)

header {}
	language_version {}
	author {}
	date_creation {}
	date_change {}
	description {}
units {}
datatypes {}
	datatype {'description': 'predefined ADTF tBool datatype', 'name': 'tBool', 'size': '8'}
	datatype {'description': 'predefined ADTF tChar datatype', 'name': 'tChar', 'size': '8'}
	datatype {'description': 'predefined ADTF tUInt8 datatype', 'name': 'tUInt8', 'size': '8'}
	datatype {'description': 'predefined ADTF tInt8 datatype', 'name': 'tInt8', 'size': '8'}
	datatype {'description': 'predefined ADTF tUInt16 datatype', 'name': 'tUInt16', 'size': '16'}
	datatype {'description': 'predefined ADTF tInt16 datatype', 'name': 'tInt16', 'size': '16'}
	datatype {'description': 'predefined ADTF tUInt32 datatype', 'name': 'tUInt32', 'size': '32'}
	datatype {'description': 'predefined ADTF tInt32 datatype', 'name': 'tInt32', 'size': '32'}
	datatype {'description': 'predefined ADTF tUInt64 datatype', 'name': 'tUInt64', 'size': '64'}
	datatype {'description': 'predefined ADTF tInt64 dat

In [3]:
dtypes=[]
for a in root.findall('datatypes'):
    for b in a.getchildren():
        #print( b.get('description').split('typet')[-1] )
        #print(b.get('description'), end=' ')
        dtypes.append(b.get('name'))
        print(b.get('name'))

tBool
tChar
tUInt8
tInt8
tUInt16
tInt16
tUInt32
tInt32
tUInt64
tInt64
tFloat32
tFloat64


In [4]:
sDic = {
    'tBool':   '?',
    'tChar':   's',
    'tUInt8':  'B', #unsigned char with size 1
    'tInt8':   'b', #signed char 1 byte (8bits)
    'tUInt16': 'H', #unsigned short integer 2 bytes
    'tInt16':  'h', 
    'tUInt32': 'I', #4 bytes
    'tInt32':  'i',
    'tUInt64': 'Q',
    'tInt64':  'q',
    'tFloat32':'f', #float 4bytes
    'tFloat64':'d', #double
    'tPolarCoordiante': 'ff', #TODO
    'tSignalValue': 'If', #TODO
}

In [5]:
# something missing?
set(dtypes)-sDic.keys()

set()

In [6]:
# go trought elements of structs and save it as DataFrame
structs = {}
for struct in root.find('structs').getchildren():
    elms=[]
    for b in struct.getchildren():
        #print(b.attrib)
        elms.append(b.attrib)
    structs[struct.get('name')]=pd.DataFrame(elms)
structs.keys()

dict_keys(['tJuryStruct', 'tDriverStruct', 'tSignalValue', 'tBoolSignalValue', 'tWheelData', 'tInerMeasUnitData', 'tRoadSignExt', 'tPosition', 'tObstacle', 'tTrafficSign', 'tParkingSpace', 'tUltrasonicStruct', 'tVoltageStruct', 'tPolarCoordiante', 'tLaserScannerData', 'tClassification', 'tVirtualPoint'])

In [7]:
structs['tSignalValue']

,alignment,arraysize,byteorder,bytepos,name,type
0,1,1,LE,0,ui32ArduinoTimestamp,tUInt32
1,1,1,LE,4,f32Value,tFloat32


In [8]:
# check again if there are any structs missing in the sDic
types = []
for k in structs.keys():
    for e in list(structs[k].type):
        types.append(e)
types=[*types]
types=set(types)
types-sDic.keys() #stuff that is not in sDic

set()

In [9]:
# add a 'format' column to the structs
for k in structs.keys():
    structs[k]['format']=structs[k]['type'].apply(lambda x: sDic[x])
    #not that easy --> has to resolve master type first
    
    #TODO --> try catch..

In [10]:
def getFormat(key):
    return ''.join(list(structs[key].format))
getFormat('tPolarCoordiante')

'ff'

In [84]:
# one could also use the overall dictionary to search there ;)
formatDict={}
for k in structs.keys():
    formatDict[k]=getFormat(k)
formatDict

{'tJuryStruct': 'bh',
 'tDriverStruct': 'hh',
 'tSignalValue': 'If',
 'tBoolSignalValue': 'I?',
 'tWheelData': 'IIb',
 'tInerMeasUnitData': 'Ifffffffff',
 'tRoadSignExt': 'hfff',
 'tPosition': 'fffff',
 'tObstacle': 'ff',
 'tTrafficSign': 'hfff',
 'tParkingSpace': 'hffH',
 'tUltrasonicStruct': 'IfIfIfIfIf',
 'tVoltageStruct': 'IfIfIfIfIfIfIfIfIfIf',
 'tPolarCoordiante': 'ff',
 'tLaserScannerData': 'Iff',
 'tClassification': 'sQd',
 'tVirtualPoint': 'dddd'}

# try it out

In [112]:
import struct
def parseZMQ(data,data_types):
    """
        Parses the ZMQ data given the data-types and the data in form of a list
        Returns a list
    """
    parsed_data = []
    for i, data_type in enumerate(data_types):
        parsed_data.append( struct.unpack(formatDict[data_type], data[i]) )
    return parsed_data

def getDataTypeColumnName(data_type):
    """
        Returns the column names for a data-type
    """
    columns=[]
    for tpe,name in zip(structs[data_type].type.values, structs[data_type].name.values):
        if tpe in structs.keys():
            for n in structs[tpe].name.values:
                columns.append(n)
        else:
            columns.append(name)
    return columns

def parseToZMQ(data,data_type):
    """
        Parses one 
        Returns
    """
    parsed_data = struct.pack(formatDict[data_type], *data)
    return parsed_data


In [90]:
cNameDict={}
for k in structs.keys():
    cNameDict[k]=getDataTypeColumnName(k)

In [87]:
cNameDict['tUltrasonicStruct'], formatDict['tUltrasonicStruct']

(['ui32ArduinoTimestamp',
  'f32Value',
  'ui32ArduinoTimestamp',
  'f32Value',
  'ui32ArduinoTimestamp',
  'f32Value',
  'ui32ArduinoTimestamp',
  'f32Value',
  'ui32ArduinoTimestamp',
  'f32Value'],
 'IfIfIfIfIf')

In [67]:
import test_data
data_types = ['tInerMeasUnitData','tUltrasonicStruct','tUltrasonicStruct']
parsed_data =[]
for i in range(len(test_data.data)):
    parsed_data.append( parseZMQ(test_data.data[0],data_types) )

In [70]:
import test_data3

data_types = [ 'tInerMeasUnitData','tUltrasonicStruct','tWheelData','tVoltageStruct']

parsed_data =[]
for i in range(len(test_data3.data)):
    parsed_data.append( parseZMQ(test_data3.data[0],data_types) )

In [116]:
data_type='tSignalValue'
[parseToZMQ([516556, 9999],data_type), parseToZMQ([516558,70135],data_type)]

[b'\xcc\xe1\x07\x00\x00<\x1cF', b'\xce\xe1\x07\x00\x80\xfb\x88G']

# connect to socket

In [96]:
import zmq
import zmqParser

# connect to the car
context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:5555")

# edit data_types
ddTps = [ 'tInerMeasUnitData','tUltrasonicStruct','tWheelData','tVoltageStruct']

while True:
    bin_msg = socket.recv_multipart()
    msg = zmqParser.parseFromZMQ(bin_msg, ddTyps)
    print(msg)
    
    # send same stuff again
    ddType = 'tSignalValue'
    bin_msg = zmqParser.parseToZMQ(msg, ddTyp)
    socket.send_multipart(bin_msg)
    
socket.close()

ZMQError: Socket operation on non-socket